In [ ]:
# Package imports
import numpy as np
import sklearn
import sklearn.linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
import pandas as pd

%matplotlib inline

np.random.seed(777)

In [ ]:
df = pd.read_csv('Vehicles.csv',header = None)
print(df.isnull().values.any())

scaler = preprocessing.StandardScaler().fit(df.drop(18,axis =1))
scaled_features = scaler.transform(df.drop(18,axis =1))
df_features = pd.DataFrame(scaled_features,columns = df.columns[:-1])

df_label = df[df.columns[-1]]

#df[df.columns[len(df.columns)-1]]

df_features.head()

In [ ]:
X = df_features
y = df_label.values.reshape(-1,1)


train_data, test_data, train_labels, test_labels = train_test_split(X,y,test_size=0.30,random_state=777)
val_data, test_data, val_labels, test_labels = train_test_split(test_data,test_labels ,test_size=0.50,random_state=777)


In [ ]:
print(train_labels.shape)

In [ ]:
ohe = OneHotEncoder()
train_labels = train_labels.reshape(len(train_labels),1)
train_labels = ohe.fit_transform(train_labels)

test_labels = test_labels.reshape(len(test_labels),1)
test_labels = ohe.fit_transform(test_labels)

val_labels = val_labels.reshape(len(val_labels),1)
val_labels = ohe.fit_transform(val_labels)

In [ ]:
trainx = train_data.T
trainy = train_labels.T

valx = val_data.T
valy = val_labels.T

testx = test_data.T
testy =test_labels.T


In [ ]:
trainx.shape, trainy.shape, valx.shape, valy.shape, testx.shape, testy.shape

In [ ]:
X=trainx
Y=trainy

In [ ]:

### START CODE HERE ###
shape_X = trainx.shape
shape_Y = trainy.shape
m = trainx.shape[1]  # training set size
### END CODE HERE ###

print ('No. of training samples: ' + str(m))
print ('Number of features per sample: ' + str(shape_X[0]))


In [ ]:
def model_architecture(X, Y):
    """
    Arguments:
    X -- input dataset of shape (input size, number of examples)
    Y -- labels of shape (output size, number of examples)
    
    Returns:
    n_x -- the size of the input layer
    n_h -- the size of the hidden layer
    n_y -- the size of the output layer
    """
    ### START CODE HERE ### 
    n_x = X.shape[0] # size of input layer
    n_h = 10
    n_y = Y.shape[0] # size of output layer
    ### END CODE HERE ###
    return (n_x, n_h, n_y)

In [ ]:
def initialize_parameters(n_x, n_h, n_y):

    np.random.seed(777)

    
    ### START CODE HERE ###
    W1 = np.random.randn(n_h,n_x)*0.01
    b1 = np.zeros((n_h,1))
    W2 = np.random.randn(n_y,n_h)*0.01
    b2 = np.zeros((n_y,1))
    ### END CODE HERE ###
    
    assert (W1.shape == (n_h, n_x))
    assert (b1.shape == (n_h, 1))
    assert (W2.shape == (n_y, n_h))
    assert (b2.shape == (n_y, 1))
    
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    return parameters

In [ ]:
def softmax(x):
    ### Update THE CODE HERE ###
    
    exp_x = np.exp(x)
    return exp_x/np.sum(exp_x)   
    ### END CODE HERE ###

In [ ]:
def forward_propagation(X, parameters):
    
    # Retrieve each parameter from the dictionary "parameters"
    ### START CODE HERE ### 
    W1 = parameters["W1"]
    b1 = parameters["b1"]
    W2 = parameters["W2"]
    b2 = parameters["b2"]
    ### END CODE HERE ###
    
    # Implement Forward Propagation to calculate A2 (probabilities)
    ### START CODE HERE ### 
    Z1 = np.dot(W1 , X) + b1
    A1 = np.tanh(Z1)
    Z2 = np.dot(W2 , A1) + b2
    A2 = softmax(Z2)
    
    ### END CODE HERE ###
    

    
    cache = {"Z1": Z1,
             "A1": A1,
             "Z2": Z2,
             "A2": A2}
    
    return A2, cache

In [ ]:
def compute_loss(A2, Y):
   
    m = Y.shape[1] # number of example
    
    
    cost = -(1/m)*np.sum(np.multiply(Y,np.log(A2)))
    loss= np.sum(cost)/(100*m)
    

    return loss

In [ ]:
def backprop(parameters, cache, X, Y):

    m = X.shape[1]
    
    # First, retrieve W1 and W2 from the dictionary "parameters".
    ### START CODE HERE ### 
    W1 = parameters["W1"]
    b1 = parameters["b1"]
    W2 = parameters["W2"]
    b2 = parameters["b2"]
    ### END CODE HERE ###
        
    # Retrieve also A1 and A2 from dictionary "cache".
    ### START CODE HERE ### 
    A1 = cache["A1"]
    A2 = cache["A2"]
    ### END CODE HERE ###
    
    # Backward propagation: calculate dW1, db1, dW2, db2. 
    ### START CODE HERE ### 
    dZ2 = A2 - Y
    dW2 = (1/m)*np.dot(dZ2,A1.T)
    db2 = (1/m)*np.sum(dZ2)
    dZ1 = np.multiply(np.dot(W2.T,dZ2), 1-np.power(A1,2))
    dW1 = (1/m)*np.dot(dZ1,X.T)
    db1 = (1/m)*np.sum(dZ1)
    ### END CODE HERE ###
    
    grads = {"dW1": dW1,
             "db1": db1,
             "dW2": dW2,
             "db2": db2}
    
    return grads

In [ ]:
def update(parameters, grads, learning_rate = 0.01):

    # Retrieve each parameter from the dictionary "parameters"
    ### START CODE HERE ### 
    W1 = parameters["W1"]
    b1 = parameters["b1"]
    W2 = parameters["W2"]
    b2 = parameters["b2"]
    ### END CODE HERE ###
    
    # Retrieve each gradient from the dictionary "grads"
    ### START CODE HERE ### 
    dW1 = grads["dW1"]
    db1 = grads["db1"]
    dW2 = grads["dW2"]
    db2 = grads["db2"]
    ## END CODE HERE ###
    
    # Update rule for each parameter
    ### START CODE HERE ### 
    W1 = W1 - learning_rate*dW1
    b1 = b1 - learning_rate*db1
    W2 = W2 - learning_rate*dW2
    b2 = b2 - learning_rate*db2
    ### END CODE HERE ###
    
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    return parameters

In [ ]:
def NeuralNetwork(X, Y, n_h, num_iterations = 10000, learning_rate = 0.01, print_loss=False):

    np.random.seed(3)
    n_x = model_architecture(X, Y)[0]
    n_y = model_architecture(X, Y)[2]
    
    # Initialize parameters
    ### START CODE HERE ### 
    parameters = initialize_parameters(n_x, n_h, n_y)
    W1 = parameters["W1"]
    b1 = parameters["b1"]
    W2 = parameters["W2"]
    b2 = parameters["b2"]
    ### END CODE HERE ###
    
    # Loop (gradient descent)

    for i in range(0, num_iterations):
         
        ### START CODE HERE ### 
        # Forward propagation. Inputs: "X, parameters". Outputs: "A2, cache".
        A2, cache = forward_propagation(X, parameters)
        
        # loss function. Inputs: "A2, Y, parameters". Outputs: "loss".
        loss = compute_loss(A2, Y)
 
        # Backpropagation. Inputs: "parameters, cache, X, Y". Outputs: "grads".
        grads = backprop(parameters, cache, X, Y)
 
        # Gradient descent parameter update. Inputs: "parameters, grads". Outputs: "parameters".
        parameters =  update(parameters, grads, learning_rate = 0.01)
        
        ### END CODE HERE ###
        
        # Print the loss every 100 iterations
        if print_loss and i % 100 == 0:
            
            print ("loss after iteration %i: %f" %(i, loss))

    return parameters

In [ ]:
# Build a model with a n_h-dimensional hidden layer
parameters = NeuralNetwork(X, Y, n_h = 10, num_iterations = 10000, learning_rate = 0.1, print_loss=True)

In [ ]:
def predict(parameters, X):

    # Computes probabilities using forward propagation, and classifies to 0/1 using 0.5 as the threshold.
    ### START CODE HERE ### 1
    A2, cache = forward_propagation(X, parameters)
    predictions = np.argmax(A2, 0)
    
    ### END CODE HERE ###
    
    return predictions

In [ ]:
# Print accuracy
predictions = predict(parameters, trainx)
y_train = np.argmax(trainy, 0)

accuracy = np.mean(y_train == predictions)*100
print('train accuracy = ',accuracy)


In [ ]:
predictions = predict(parameters, testx)
y_test = np.argmax(testy, 0)

accuracy = np.mean(y_test == predictions)*100
print('test accuracy = ',accuracy)

In [ ]:
from sklearn.metrics import confusion_matrix

#Generate the confusion matrix

print(np.ravel(y_train).shape)

qw = np.ravel(y_train)
qs = qw[:127]
print(np.ravel(predictions).shape)

cf_matrix = confusion_matrix(qs, np.ravel(predictions))
print(cf_matrix)

In [ ]:
import seaborn as sns
sns.heatmap(cf_matrix, annot=True)